In [3]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import urllib2
import sqlite3

url = "http://db.netkeiba.com/race/201606050903/"
soup = BeautifulSoup(urllib2.urlopen(url), "html.parser")


In [24]:
dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()

drop_table = '''drop table race'''
c.execute(drop_table)
drop_table = '''drop table race_by_horse'''
c.execute(drop_table)
drop_table = '''drop table horse'''
c.execute(drop_table)
drop_table = '''drop table past_race'''
c.execute(drop_table)

c.close()

In [25]:
dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()

create_table = '''create table race (race_id integer PRIMARY_KEY, name varchar(64)
                  )'''
c.execute(create_table)

create_table = '''create table race_by_horse (id integer PRIMARY KEY, horse_id int, race_id int,
                  finished int, waku_ban int, uma_ban int, name varchar(64),
                  sex int, age int, jockey_weight int, jockey_id int, jockey_name varchar(64),
                  time varchar(64), odds, popularity int, weight int)'''
c.execute(create_table)

create_table = '''create table horse (horse_id integer PRIMARY KEY, name varchar(64),
                  age int, gender varchar(32))'''
c.execute(create_table)

create_table = '''create table past_race (past_race_id integer PRIMARY KEY, horse_id int,
                  date varchar(64), place varchar(64), race_name varchar(64),
                  num_horses int, waku_ban int, uma_ban int, odds, popularity int,
                  finished int, jockey_weight int, jockey_id int, jockey_name varchar(64),
                  time varchar(64), weight int)'''
c.execute(create_table)
c.close()

In [26]:
# -*- coding: utf-8 -*-
main_table = soup.find("table", {"class":"race_table_01"})
tr_list = main_table.find_all("tr")

row_list = []
horse_list = []

for i, tr in enumerate(tr_list):
    if i == 0:
        continue
    td = tr.find("td")
    finished = td.text
    td = td.findNext("td")
    waku_ban= td.text
    td = td.findNext("td")
    uma_ban = td.text
    td = td.findNext("td")
    name = td.text.strip()
    td = td.findNext("td")
    sex_age = td.text
    sex = list(sex_age)[0]
    age = list(sex_age)[1]
    td = td.findNext("td")
    jockey_weight = td.text
    td = td.findNext("td")
    jockey_name = td.text.strip()
    td = td.findNext("td")
    time = td.text
    #diff, timeindex, passed, halftime is skipped
    td = td.findNext("td").findNext("td").findNext("td").findNext("td")
    td = td.findNext("td")
    odds = td.text
    td = td.findNext("td")
    popularity = td.text
    td = td.findNext("td")
    weight= td.text.split("(")[0]
    row = (finished, waku_ban, uma_ban, name, sex, age,
          jockey_weight, jockey_name, unicode(time), odds, popularity, weight)
    row_list.append(row)
print len(row_list)

16


In [27]:
# -*- coding: utf-8 -*-
dbname = 'database.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()

insert_sql = """insert into race_by_horse (finished, waku_ban, uma_ban, name, sex, age,
          jockey_weight, jockey_name, time, odds, popularity, weight)
          values (?,?,?,?,?,?,?,?,?,?,?,?)"""
c.executemany(insert_sql, row_list)
conn.commit()

conn.close()

In [29]:
# -*- coding: utf-8 -*-
dbname = 'database.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()

param =(u"ダーナ",)
select_sql ="select * from race_by_horse where name = ?"
results = c.execute(select_sql, param)
for res in results:
    print "-------"
    for r in res:
        print r,

c.close()

-------
12 None None 12 2 3 ダーナ 牝 2 52 None 原田和真 1:38.8 14.5 6 390


In [62]:
dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()
c.close()